# Importing Packages

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, preprocessing, svm
from sklearn.preprocessing import StandardScaler, Normalizer
import math
import matplotlib
import seaborn as sns

%matplotlib inline

### Useful functions

In [ ]:
def category_values(dataframe, categories):
    for c in categories:
        print('\n', dataframe.groupby(by=c)[c].count().sort_values(ascending=False))
        print('Nulls: ', dataframe[c].isnull().sum())

def plot_correlation_map( df ):
    corr = df.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )



# Preparing data

## Reading from file

In [ ]:
df = pd.read_csv('autos.csv', sep=',', header=0, encoding='cp1252')
#df = pd.read_csv('autos.csv.gz', sep=',', header=0, compression='gzip',encoding='cp1252')
df.sample(10)

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
33117,2016-03-19 21:36:21,Schoener_Golf_Variant,privat,Angebot,18700,control,NaN,2017,manuell,150,golf,30000,6,NaN,volkswagen,nein,2016-03-19 00:00:00,0,26802,2016-03-23 12:19:44
289080,2016-03-19 07:54:21,BMW_e36_316i,privat,Angebot,800,control,limousine,1993,manuell,101,NaN,150000,5,benzin,bmw,nein,2016-03-19 00:00:00,0,86836,2016-04-06 06:16:38
293451,2016-03-31 18:47:02,Audi_A6_2.5TDI___defekt,privat,Angebot,800,control,limousine,2001,manuell,149,a6,150000,4,diesel,audi,ja,2016-03-31 00:00:00,0,55595,2016-03-31 18:47:02
10397,2016-03-11 12:47:58,BMW_330d_DPF__XENON__HIFI__SPORTSITZE__M_FAHRWERK,privat,Angebot,13000,test,limousine,2011,manuell,245,3er,150000,4,diesel,bmw,nein,2016-03-11 00:00:00,0,50739,2016-04-07 06:45:02
37554,2016-03-29 22:47:57,Mercedes_Benz_E_320,privat,Angebot,1700,test,limousine,1995,automatik,220,e_klasse,150000,11,benzin,mercedes_benz,nein,2016-03-29 00:00:00,0,32758,2016-04-06 10:45:36
262635,2016-03-08 10:56:31,Mercedes_Benz_R_300_CDI_DPF_BlueEFFICIENCY_7G_...,privat,Angebot,32850,test,bus,2012,automatik,190,andere,60000,12,diesel,mercedes_benz,nein,2016-03-08 00:00:00,0,10707,2016-03-30 17:15:23
93140,2016-03-22 19:53:22,Volkswagen_Golf_Variant_1.9_TDI_Edition,privat,Angebot,1950,test,kombi,2000,manuell,90,golf,150000,8,diesel,volkswagen,nein,2016-03-22 00:00:00,0,30455,2016-03-24 04:46:46
23619,2016-03-19 18:54:12,Smart_smart_&_pure,privat,Angebot,1199,test,kleinwagen,1999,manuell,45,fortwo,150000,10,benzin,smart,nein,2016-03-19 00:00:00,0,94148,2016-04-07 06:16:37
182367,2016-03-08 04:00:04,VW_Passat_1_6_FSI_Baujahr_10/2006,privat,Angebot,5500,control,limousine,2006,manuell,116,passat,150000,10,benzin,volkswagen,nein,2016-03-08 00:00:00,0,46537,2016-03-09 06:16:58
172621,2016-03-07 21:52:37,Mein_geliebtes_Kadett_GSI_Cabrio,privat,Angebot,3000,control,cabrio,1991,manuell,150,kadett,150000,7,benzin,opel,nein,2016-03-07 00:00:00,0,12627,2016-04-06 03:45:51


In [ ]:
df.describe()

,price,yearOfRegistration,powerPS,kilometer,monthOfRegistration,nrOfPictures,postalCode
count,3.715280e+05,371528.000000,371528.000000,371528.000000,371528.000000,371528.0,371528.00000
mean,1.729514e+04,2004.577997,115.549477,125618.688228,5.734445,0.0,50820.66764
std,3.587954e+06,92.866598,192.139578,40112.337051,3.712412,0.0,25799.08247
min,0.000000e+00,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.00000
25%,1.150000e+03,1999.000000,70.000000,125000.000000,3.000000,0.0,30459.00000
50%,2.950000e+03,2003.000000,105.000000,150000.000000,6.000000,0.0,49610.00000
75%,7.200000e+03,2008.000000,150.000000,150000.000000,9.000000,0.0,71546.00000
max,2.147484e+09,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.00000


# Dropping some useless columns


In [ ]:
print(df.seller.unique())
print(df.offerType.unique())
print(df.abtest.unique())
print(df.nrOfPictures.unique())

['privat' 'gewerblich']
['Angebot' 'Gesuch']
['test' 'control']
[0]


In [ ]:
df.drop(['seller', 'offerType', 'abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen', 'postalCode', 'dateCreated'], axis='columns', inplace=True)


## Cleaning data

Cleaning data from duplicates, NaNs and selecting reasonable ranges for columns


In [ ]:
print("Too new: %d" % df.loc[df.yearOfRegistration >= 2017].count()['name'])
print("Too old: %d" % df.loc[df.yearOfRegistration < 1950].count()['name'])
print("Too cheap: %d" % df.loc[df.price < 100].count()['name'])
print("Too expensive: " , df.loc[df.price > 150000].count()['name'])
print("Too few km: " , df.loc[df.kilometer < 5000].count()['name'])
print("Too many km: " , df.loc[df.kilometer > 200000].count()['name'])
print("Too few PS: " , df.loc[df.powerPS < 10].count()['name'])
print("Too many PS: " , df.loc[df.powerPS > 500].count()['name'])
print("Fuel types: " , df['fuelType'].unique())
#print("Offer types: " , df['offerType'].unique())
#print("Sellers: " , df['seller'].unique())
print("Damages: " , df['notRepairedDamage'].unique())
#print("Pics: " , df['nrOfPictures'].unique()) # nrOfPictures : number of pictures in the ad (unfortunately this field contains everywhere a 0 and is thus useless (bug in crawler!) )
#print("Postale codes: " , df['postalCode'].unique())
print("Vehicle types: " , df['vehicleType'].unique())
print("Brands: " , df['brand'].unique())

# Cleaning data
#valid_models = df.dropna()

#### Removing the duplicates
dedups = df.drop_duplicates(['name','price','vehicleType','yearOfRegistration'
                         ,'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType'
                         ,'notRepairedDamage'])

#### Removing the outliers
dedups = dedups[
        (dedups.yearOfRegistration <= 2016) 
      & (dedups.yearOfRegistration >= 1950) 
      & (dedups.price >= 100) 
      & (dedups.price <= 150000) 
      & (dedups.powerPS >= 10) 
      & (dedups.powerPS <= 500)]

print("-----------------\nData kept for analisys: %d percent of the entire set\n-----------------" % (100 * dedups['name'].count() / df['name'].count()))


Too new: 14680
Too old: 289
Too cheap: 13320
Too expensive:  232
Too few km:  0
Too many km:  0
Too few PS:  41040
Too many PS:  835
Fuel types:  ['benzin' 'diesel' nan 'lpg' 'andere' 'hybrid' 'cng' 'elektro']
Damages:  [nan 'ja' 'nein']
Vehicle types:  [nan 'coupe' 'suv' 'kleinwagen' 'limousine' 'cabrio' 'bus' 'kombi'
 'andere']
Brands:  ['volkswagen' 'audi' 'jeep' 'skoda' 'bmw' 'peugeot' 'ford' 'mazda'
 'nissan' 'renault' 'mercedes_benz' 'opel' 'seat' 'citroen' 'honda' 'fiat'
 'mini' 'smart' 'hyundai' 'sonstige_autos' 'alfa_romeo' 'subaru' 'volvo'
 'mitsubishi' 'kia' 'suzuki' 'lancia' 'porsche' 'toyota' 'chevrolet'
 'dacia' 'daihatsu' 'trabant' 'saab' 'chrysler' 'jaguar' 'daewoo' 'rover'
 'land_rover' 'lada']
-----------------
Data kept for analisys: 81 percent of the entire set
-----------------


## Working on the `null` values

Checking if theree are NaNs to fix or drop

In [ ]:
dedups.isnull().sum()

name                       0
price                      0
vehicleType            10818
yearOfRegistration         0
gearbox                 5260
powerPS                    0
model                  11347
kilometer                  0
monthOfRegistration        0
fuelType               15400
brand                      0
notRepairedDamage      42124
dtype: int64

In [ ]:
dedups['notRepairedDamage'].fillna(value='not-declared', inplace=True)
dedups['fuelType'].fillna(value='not-declared', inplace=True)
dedups['gearbox'].fillna(value='not-declared', inplace=True)
dedups['vehicleType'].fillna(value='not-declared', inplace=True)
dedups['model'].fillna(value='not-declared', inplace=True)

Checking if all the nulls have been filled or dropped.

In [ ]:
dedups.isnull().sum()

name                   0
price                  0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64